### Add libraries & read the image

In [3]:
import cv2
import numpy as np

img = cv2.imread("bioimage.png")
img_height = img.shape[0]
img_width = img.shape[1]
print(img_width, img_height)

622 962


### Show the image

In [ ]:
cv2.imshow("img",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Converting image to blob
##### What is blob? Images converted to tensors with 4 dimension

In [5]:
img_blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), swapRB=True,crop=False) #params: img resource, scale factorm, blob dimensions(2),BGR to RGB,to crop?
print(img_blob.shape)

(1, 3, 416, 416)


### Read and split labels

In [39]:
label_file = open("labels.txt", 'r')
labels = [word.replace('"','') for word in str(label_file.readlines()).split(',')]
print(len(labels),labels)

80 ["['person", 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'trafficlight', 'firehydrant', 'stopsign', 'parkingmeter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sportsball', 'kite', 'baseballbat', 'baseballglove', 'skateboard', 'surfboard', 'tennisracket', 'bottle', 'wineglass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hotdog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cellphone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddybear', 'hairdrier', "toothbrush']"]


### Create color list for bounding boxes

In [38]:
colors = ["0,255,0","0,0,255","255,0,0","0,120,30","0,30,120","50,50,50","50,0,50","50,50,100"]
colors = [np.array(color.split(",")).astype("int") for color in colors]
colors = np.array(colors)
colors = np.tile(colors,(10,1)) #copying color list 10 times to fill the array with same numbers vertically
len(colors)

80

### Read the model and layers

In [52]:
model = cv2.dnn.readNetFromDarknet("model/yolov3.cfg","model/yolov3.weights")
layers = model.getLayerNames()
output_layers = [layers[int(layer)-1] for layer in model.getUnconnectedOutLayers()]

In [53]:
model.setInput(img_blob)
detection_layers = model.forward(output_layers) # reaching out to the values in output layers
detection_layers

[array([[0.04120015, 0.04584586, 0.37053925, ..., 0.        , 0.        ,
         0.        ],
        [0.04649337, 0.03197153, 0.27123636, ..., 0.        , 0.        ,
         0.        ],
        [0.04628211, 0.03658194, 0.8056842 , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.9572996 , 0.9502675 , 0.4856819 , ..., 0.        , 0.        ,
         0.        ],
        [0.9624371 , 0.96359324, 0.3226212 , ..., 0.        , 0.        ,
         0.        ],
        [0.9719613 , 0.96827793, 0.8332815 , ..., 0.        , 0.        ,
         0.        ]], dtype=float32),
 array([[0.01874625, 0.02464668, 0.05372084, ..., 0.        , 0.        ,
         0.        ],
        [0.01951553, 0.01811345, 0.39146778, ..., 0.        , 0.        ,
         0.        ],
        [0.02242082, 0.01782617, 0.07499409, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.977514  , 0.97652143, 0.03970766, ..., 0.        , 0.        ,
         0.        